In [44]:
import os, sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

# from itables import init_notebook_mode
# init_notebook_mode(all_interactive=True)
# --- 
# from techminer2 import vantagepoint

import pandas as pd


df = pd.read_csv('dyna/processed/_documents.csv')

authors = df[['authors']].dropna()
for x in authors[authors.authors.str.contains("oz B")].authors.to_list():
    print(x)


de Oliveira JD; Biondi D; Reis ARN; Viezzer J; oz B
Leon RAG; Afanador-Garcia N; Guerrero-Gomez G; oz B
Avila-Lara DD; Rubio-Rios A; Rosales-Marines L; Solanilla-Duque JF; Flores-Gallegos AC; Rodriguez-Herrera R; oz B
Montoya-Castrillon M; Serna-Vasco KJ; Pinilla L; Quiceno-Rico JM; Cardona-Bermudez LM; Osorio-Echavarria J/1; oz B
Garrido-Tellez JL; Hernandez-Martinez EE; Jimenez-Escalona JC; oz B
Almeida TAB; de Assuncao Montenegro AA; de Carvalho AA; Tabosa JN; oz B
Gomez-Garcia CA; Velasco-Medina J; oz B
Perez-Martinez L; Naranjo-Rey MA; Santos-Perez O; Cabrera-Hernandez JA; Nogueira-Rivera D; oz B
Martinez-Conde JM; Patino-Vanegas A; oz B
Galvis-Velasco LC; Ruge-Cardenas JC; Galvis-Salamanca LCA; Pulgarin-Morales L; Bastidas-Martinez JG; Olarte MC; oz B
Borroto-Penton Y; Caraza-Morales MA; Alfonso-Llanes A; Marrero-Delgado F; oz B
Socarras-Cordovi YC; Alvarez-Deulofeu E; Pupo-Sintras N; oz B
Apolo-Matamoros EO; Rueda WJG; oz B
Meza-Salazar G; Cruz-Manjarrez H; Agredo-Diaz DG; Ortiz